## Romiumeter: Validation of accuracy and reliablity of romiumeter in computing range of motion

Accuracy validation of romiumeter is against optitrack motion caputure system.

Algorithms that are validated:
1. Gyroscope integration
2. Complementary Filter
3. Madgwick Filter


**Standard Modules**

In [1]:
%matplotlib inline
import mpld3
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np 
import math
from scipy import interpolate
from scipy.signal import savgol_filter
import csv
mpld3.enable_notebook()
from matplotlib.ticker import FormatStrFormatter
import pathlib
from scipy.stats import f_oneway
import glob

**Custom Functions**

In [2]:
def get_the_data_path(data_dir):
    """get the path of all the files
    """
    data_path = []
    sub_dir1 = [os.path.join(data_dir, file) for file in os.listdir(data_dir)]
    for file_1 in sub_dir1:
        sub_dir2 = [os.path.join(file_1, file) for file in os.listdir(file_1)]
        for file_2 in sub_dir2:
            sub_dir3 = [os.path.join(file_2, file) for file in os.listdir(file_2)]
            for file_3 in sub_dir3:
                sub_dir4 = [os.path.join(file_3, file) for file in os.listdir(file_3)]
                for file_4 in sub_dir4:
                    sub_dir5 = [os.path.join(file_4, file) for file in os.listdir(file_4)]
                    for file_5 in sub_dir5:
                        _data_path = [os.path.join(file_5, file) for file in os.listdir(file_5)]
                        data_path.append(_data_path)
    return data_path
def get_the_enc_data_path(data_dir):
    """
    get the path of all the files
    """
    data_path = []
    sub_dir1 = [os.path.join(data_dir, file) for file in os.listdir(data_dir)]         
    for file_1 in sub_dir1:
        _data_path = [os.path.join(file_1, file) for file in os.listdir(file_1)]
        data_path.append(_data_path)
    return data_path
def get_the_imu_mcp_enc_data_path(data_dir):
    """
    get the path of all the files
    """
    imu_data_path = []
    mcp_data_path = []
    sub_dir1 = [os.path.join(data_dir, file) for file in os.listdir(data_dir)]     
    for file in sub_dir1:
        if file.find('imu') != -1:
            _data_path = [os.path.join(file, file1) for file1 in os.listdir(file)]
            imu_data_path.append(_data_path)
        elif file.find('mcp') != -1:
            _data_path = [os.path.join(file, file1) for file1 in os.listdir(file)]
            mcp_data_path.append(_data_path)
    return imu_data_path, mcp_data_path
def upsample_mocap_data(imu: pd.DataFrame, mcp: pd.DataFrame) -> pd.DataFrame:
     """upsample the mocap data to match the sampling frequency of imu data.

     Parameters
     ----------
     imu: pandas.DataFrame 
          DataFrame consisting of time, accelerometer and gyroscope data
     mcp: pandas.Dataframe
          DataFrame consisting of time, position of markers

     Returns
     -------
     pandas.DataFrame
          Returns DataFrame consisting upsampled motion capture data consisting
          of time, position of markers.
     """
    
     _t = imu.Time
     _anew = pd.DataFrame()
     _anew['Time'] = _t
     for col in mcp.columns:
          f = interpolate.interp1d(mcp.Time,mcp[col],fill_value="extrapolate", kind = 'cubic')
          _anew[col]=f(_t)
     mcp=_anew
     return mcp
def read_filter_data(fname: str, datacols: list, filtcols: list, sgwin: int, sgorder: int) -> pd.DataFrame:
    """read the comma-seperated values(csv) file and low pass filter the selected columns

    Parameters
    ----------
    fname:    str
              A comma-seperated vlalues(csv) file.
    datacols: list
              list of columns in the csv file.
    filtcols: list
              list of columsn in the csv file to be filterd.
    sgwin:    int
              The length of the filter window
    sgorder:  int
              The order of the polynomial used to fit the samples

    Returns
    -------
    pandas.DataFrame
        Returns DataFrame with the selected data of selected columsn low pass filtered.
    """
    _data = pd.read_csv(fname, usecols=datacols)
    for fc in filtcols:
        _data[fc] = savgol_filter(_data[fc], sgwin, sgorder)
    return _data
def imu_calibration(imu, acc_A, acc_bias, gyr_bias, scale):
    """perform the calibration of IMU 
    """
    acc = imu[["accelx1", "accely1", "accelz1"]].values
    gyro = imu[["gyrox1", "gyroy1", "gyroz1"]].values
    if scale:
        acc = acc.astype(float)/16384.0
        gyro = gyro.astype(float)/65.5
    else:
        acc = acc.astype(float)
        gyro = gyro.astype(float)
    acc -= acc_bias
    acc = acc_A @ acc.T
    _var_gyro_offst = np.var(gyro[:500],axis = 0)
    _mean_gyro_offst = np.mean(gyro[:500],axis = 0)
    if np.max(_var_gyro_offst)<1.0:
        gyro -= _mean_gyro_offst
    else:
        gyro -= gyr_bias
    return acc.T, np.deg2rad(gyro)
def gram_schmidt_orthogonalize(x: np.ndarray, y: np.ndarray) -> tuple((np.ndarray, np.ndarray)):
    """Perform the Gram-Schmidt orthogonalization.

    Parameters
    ----------
    x : np.ndarray
        2D array of number corresponding to the x vector. The columns are the
        different components, while the rows are different x vectors.

    y : np.ndarray
        2D array of number corresponding to the y vector. The columns are the
        different components, while the rows are different y vectors.

    Returns
    -------
    tuple[np.ndarray, np.ndarray]
        Returns the GS orthogonalized (x_normn, y_norm) vectors, with the rows corresponding
        to the different orthogonalized vectors, and the columsn corresponding
        to the different components.
        The norm of the rows of x_norm and y_norm will 1. The dot product of
        the rows of the x_norm nad y_norm will be 0.
    """
    assert type(x) == np.ndarray, "x must be an numpy ndarray."
    assert type(y) == np.ndarray, "x must be an numpy ndarray."
    assert len(x.shape) == 2, "x must be anb 2D numpy array."
    assert len(y.shape) == 2, "x must be anb 2D numpy array."
    assert x.shape == y.shape, "x and y must have the same shape"

    _xn = x / np.linalg.norm(x, axis=1, keepdims=True)
    _yx = np.sum(y * (_xn/np.linalg.norm(x, axis=1, keepdims=True)), axis=1, keepdims=True)
    _dy = y - x * _yx
    _yn = _dy / np.linalg.norm(_dy, axis=1, keepdims=True)
    return _xn, _yn

**Quaternion Operations**

In [3]:
def quaternion_multiply(quaternion1: np.ndarray, quaternion0: np.ndarray) -> np.ndarray:
    """Perfomre quaternion multiplication

    Parameters
    ----------
    quaternion1: np.ndarray
                 1D array of 4 numbers
    quaternion2: np.ndarray
                 1D array of 4 numbers

    Returns
    -------
    np.ndarray
        Returns 1D array of 4 numbers.
    """
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)
def inverse_of_quaternion_array(quaternion_array):
    """
    Perform the inverse of a quaternion array
    """
    inverse = np.divide(quaternion_array, np.linalg.norm(quaternion_array, axis = 1, keepdims = True))
    inverse[:, -3:] *= -1
    return inverse
def vec_quaternion(acc):
    """ 
    return vector as a quaternion with zero as first number
    """
    q_acc = np.hstack([np.zeros((len(acc),1)), acc])
    return q_acc
def quaternion_vector_rotation(quat, vec):
    """
    Perform vector rotation with quaternion
    """
    first_prod = [quaternion_multiply(quat[j], vec[j]) for j in range(len(quat))]
    rot_vect = [quaternion_multiply(first_prod[j], inverse_of_quaternion_array(quat)[j]) for j in range(len(quat))]
    return rot_vect
def quaternion_integration(quaternion,inertial_quat):
    """ 
    Perform quanternion integration
    """
    q_int = np.vstack([np.array(inertial_quat).reshape(1,4),np.zeros(((len(quaternion)-1),4))])
    for j in range(len(quaternion)-1):
        q_int[j+1] = quaternion_multiply(q_int[j],quaternion[j+1])
        q_int[j+1] = q_int[j+1]/np.linalg.norm(q_int[j+1])
    return q_int
def angle_axis_quaternion_representation(gyro,delta_t):
    """
    compute the angle axis  and quaternion representation of gyroscope data
    """
    inst_axs_of_rot = np.zeros((len(gyro), 3))
    for j in range(len(gyro)):
        if np.linalg.norm(gyro[j]) !=0 :
            inst_axs_of_rot[j] = gyro[j]/np.linalg.norm(gyro[j])
        else:
            inst_axs_of_rot[j] = gyro[j]
    delta_theta = np.linalg.norm(gyro, axis = 1)*delta_t
    q_gyro = np.column_stack([np.cos(delta_theta/2), inst_axs_of_rot*(np.sin(delta_theta/2)[:, np.newaxis])])
    return inst_axs_of_rot, delta_theta, q_gyro
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians
def euler_angles_from_quaternion_array(quaternion_array):
    """
    Get euler angles form quaternion array
    """
    tx1 = []
    ty1 = []
    tz1 = []
    for quat in quaternion_array:
        tx, ty, tz = euler_from_quaternion(quat[1], quat[2], quat[3], quat[0])
        tx1.append(tx)
        ty1.append(ty)
        tz1.append(tz)
    return tx1, ty1, tz1
def axis_of_rotation_deviation(gyro, delta_t, inertial_quat):
    """
    Compute the deviation of axis of rotation 
    """
    axs_of_rot, delta_theta, q_gyro = angle_axis_quaternion_representation(gyro,delta_t)
    q_int = quaternion_integration(q_gyro,inertial_quat)
    tx, ty, tz = euler_angles_from_quaternion_array(q_int)
    gyro_ang = 2*np.arccos(q_int[:, 0])
    ang = np.rad2deg(gyro_ang)
    index =  next(x for x, val in enumerate(ang[0:np.argmax(ang)]) if val > np.max(ang)*0.7)
    axis_of_rotation_wrt_inertial_frame = np.divide(q_int[:,1:], np.array([np.sin(gyro_ang/2), np.sin(gyro_ang/2), np.sin(gyro_ang/2)]).T)
    fixed_axis = axis_of_rotation_wrt_inertial_frame[index]/np.linalg.norm(axis_of_rotation_wrt_inertial_frame[index])
    norm_axis_of_rot = np.divide( axis_of_rotation_wrt_inertial_frame, np.linalg.norm(axis_of_rotation_wrt_inertial_frame, axis = 1, keepdims= True))
    dev_ip = norm_axis_of_rot @ fixed_axis
    dev  = np.rad2deg(np.arccos(np.clip(dev_ip, a_min= -1, a_max= 1)))
    return dev, ang, tx, ty, tz

**Notebook Level Constants**

In [4]:
imu_col =['gyrox1', 'gyroy1', 'gyroz1', 'accelx1', 'accely1', 'accelz1',  'Time']
imu_enc_col =['gyrox1', 'gyroy1', 'gyroz1', 'accelx1', 'accely1', 'accelz1', 'enc','delt','Time']
mcp_col=['Time', 'org_x', 'org_y', 'org_z', 'xvec_x', 'xvec_y', 'xvec_z', 'zvec_x', 'zvec_y', 'zvec_z']

acc_A = np.array([[2.007463, -0.000148, 0.002320],
                   [-0.000148, 1.993274, -0.000060],
                   [0.002320, -0.000060, 1.986062]])
acc_bias = np.array([0.032051, -0.009519, -0.125074])
gyr_bias = np.array([0.32372708, 1.273188007, -1.421987881])
var_lim  = 1.0
delta_t = 0.0017
inertial_quat = [1, 0, 0, 0]
alpha = 0.0000001
beta = 0.00001

header_for_gain_optmize = ['comp_1_max_abs', 'comp_2_max_abs', 'comp_3_max_abs', 'comp_4_max_abs', 'comp_5_max_abs', 'comp_6_max_abs', 'comp_7_max_abs',
                           'madg_1_max_abs', 'madg_2_max_abs', 'madg_3_max_abs', 'madg_4_max_abs', 'madg_5_max_abs', 'madg_6_max_abs', 'madg_7_max_abs',
                           'comp_1_rom_abs', 'comp_2_rom_abs', 'comp_3_rom_abs', 'comp_4_rom_abs', 'comp_5_rom_abs', 'comp_6_rom_abs', 'comp_7_rom_abs',
                           'madg_1_rom_abs', 'madg_2_rom_abs', 'madg_3_rom_abs', 'madg_4_rom_abs', 'madg_5_rom_abs', 'madg_6_rom_abs', 'madg_7_rom_abs', 
                           'gyro_max_abs', 'gyro_rom_abs']
header_final_err = ['gyro_recon_err', 'gyro_rom_err', 'comp_recon_err', 'comp_rom_err', 'madg_recon_err', 'madg_rom_err']
gain = [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
dest_key_words_imu_mcp = ['abduction', 'dummy_variab', 'e_rotation', 'i_rotation', 'dummy_variab', 'extension', 
                       'Right_Lateral_flexion', 'Left_Lateral_flexion', 'Right_rotation', 'Left_rotation']  
dest_dir = r'C:\Phase1\results_temp'
dest_file_name_imu_mcp = ['AA', 'FE', 'ER', 'IR','F','E','RLF','LLF', 'RR','LR']
dest_file_name_imu_enc = ['AA', 'FE', 'RR']
dest_key_words_imu_enc = ['abduction', 'flexion', 'rotation']
header_imu_mcp_enc = ['mcp_gyro_recon_err', 'mcp_gyro_rom_err', 'mcp_comp_recon_err', 'mcp_comp_rom_err', 'mcp_madg_recon_err', 'mcp_madg_rom_err',
                      'enc_gyro_recon_err', 'enc_gyro_rom_err', 'enc_comp_recon_err', 'enc_comp_rom_err', 'enc_madg_recon_err', 'enc_madg_rom_err']

**Angle reconstruction algorithms**

**Motion Capture System**


In [5]:
def estimate_rotation_angle_from_mocap(mcp):
    """
    Estimate joint rotation angle form motion capture data
    """
    _org_mkr = mcp.loc[:, ['org_x', 'org_y', 'org_z']].values
    _xvc_mkr = mcp.loc[:, ['xvec_x', 'xvec_y', 'xvec_z']].values
    _zvc_mkr= mcp.loc[:, ['zvec_x', 'zvec_y', 'zvec_z']].values
    gs_vec = gram_schmidt_orthogonalize(_org_mkr-_xvc_mkr, _org_mkr-_zvc_mkr)
    gs_vec = (np.cross(gs_vec[0],gs_vec[1]),) + gs_vec
    ort = np.array(gs_vec).transpose(1,0,2) 
    rot_wrt_inertial = ort[100].T @ ort
    mocap_ang = np.rad2deg(np.arccos([np.clip((np.trace(rot)-1.0)/2,a_max = 1, a_min = -1) for rot in rot_wrt_inertial]))
    return mocap_ang  

**Gyroscope Integration**

In [6]:
def angle_estimation_through_gyroscope(gyro, delta_t, inertial_quat):
        """
        Estimage joint rotation angle from gyroscope data through integration approach
        """
        axs_of_rot, delta_theta, q_gyro = angle_axis_quaternion_representation(gyro,delta_t)
        q_int = quaternion_integration(q_gyro,inertial_quat)
        gyro_ang = np.rad2deg(2*np.arccos(q_int[:, 0]))
        return gyro_ang

**Complementary Filter**

In [7]:
def tilt_correction_quaternion(q_int, acc, alpha):
    """
    Compute tilt correction quaternion
    """
    acc_rot = quaternion_vector_rotation(q_int, vec_quaternion(acc))
    acc_rot_norm = np.divide(acc_rot, np.linalg.norm(acc_rot, axis = 1, keepdims = True))
    norm_vec = np.cross(acc_rot_norm[:, [1,2,3]], acc[0].reshape(1,3))
    mag_norm_vec = np.linalg.norm(norm_vec, axis = 1, keepdims = True)
    mag_norm_vec[mag_norm_vec == 0] = 1
    axs_of_rot_acc = np.divide(norm_vec, mag_norm_vec)
    delta_theta_acc= np.arccos(np.clip(np.dot(acc_rot_norm[:, [1,2,3]], acc[0])/(np.linalg.norm(acc[0])), a_min = -1, a_max = 1.0))
    q_tilt = np.column_stack([np.cos(alpha*delta_theta_acc/2), axs_of_rot_acc*(np.sin(alpha*delta_theta_acc/2)[:, np.newaxis])])
    return q_tilt
def angle_estimation_through_complementary_filter(gyro, acc, delta_t, inertial_quat, alpha):
    """
    Estimate the joint rotation angle from IMU data through complementary filter
    """
    axs_of_rot, delta_theta, q_gyro = angle_axis_quaternion_representation(gyro,delta_t)
    q_int = quaternion_integration(q_gyro,inertial_quat)
    q_tilt = tilt_correction_quaternion(q_int, acc, alpha)
    q_corr = [np.array(quaternion_multiply(q_tilt[j], q_int[j])) for j in range(len(q_gyro))]
    q_corr_scalar = np.clip(np.array(q_corr)[:, 0], a_min = -1, a_max = 1.0)
    comp_ang = np.rad2deg(2*np.arccos(q_corr_scalar))
    return comp_ang

**Madgwick Filter**

In [8]:
def tilt_correction_estimation_through_madgwick(q_gyro, acc, delta_t, beta, inertial_quat):
    """
    Compute tilt correction through madgwick
    """
    q_corr = np.zeros((len(q_gyro),4))
    q = inertial_quat
    norm_acc = np.divide(acc, np.linalg.norm(acc, axis = 1, keepdims = True))
    for j in range(len(q_gyro)):
        f =np.array( [2*((acc[0,0]*(0.5-(q[2]*q[2])-(q[3]*q[3])))+(acc[0,1]*((q[0]*q[3])+(q[1]*q[2])))+(acc[0,2]*((q[1]*q[3])-(q[0]*q[2])))), 
                      2*((acc[0,0]*((q[1]*q[2])-(q[0]*q[3])))+(acc[0,1]*((0.5-q[1]*q[1])-(q[3]*q[3])))+(acc[0,2]*((q[0]*q[1])+(q[2]*q[3])))),
                      2*((acc[0,0]*((q[0]*q[2])-(q[1]*q[3])))+(acc[0,1]*((q[2]*q[3])-(q[0]*q[1])))+(acc[0,2]*((0.5-q[1]*q[1])-(q[2]*q[2]))))])
        f= f - norm_acc[j]
        Jac = np.array ([[(2*acc[0,1]*q[3])-(2*acc[0,2]*q[2]),(2*acc[0,1]*q[2])+(2*acc[0,2]*q[3]), ((-4*acc[0,0]*q[2])+(2*acc[0,1]*q[1])-(2*acc[0,2]*q[0])),((-4*acc[0,0]*q[3])+(2*acc[0,1]*q[0])+(2*acc[0,2]*q[1]))],
                         [(-2*acc[0,0]*q[3])+(2*acc[0,2]*q[1]),((2*acc[0,0]*q[2])-(4*acc[0,1]*q[1])+(2*acc[0,2]*q[0])),(2*acc[0,0]*q[1])+(2*acc[0,2]*q[3]), ((-2*acc[0,0]*q[0])+(-4*acc[0,1]*q[3])+(2*acc[0,2]*q[2]))],
                         [(2*acc[0,0]*q[2])-(2*acc[0,1]*q[1]), ((2*acc[0,0]*q[3])-(2*acc[0,1]*q[0])-(4*acc[0,2]*q[1])), ( (2*acc[0,0]*q[0])+(2*acc[0,1]*q[3])+(-4*acc[0,2]*q[2])),(2*acc[0,0]*q[1])+(2*acc[0,1]*q[2])]])
        del_f = Jac.T @ f
        if np.linalg.norm(del_f) != 0:
            del_f_norm = np.divide(del_f, np.linalg.norm(del_f))
        else:
            del_f_norm = del_f
        q_tilt = del_f_norm*beta*delta_t
        q = q - q_tilt                 
        q_corr[j] = q
        q = quaternion_multiply(q, q_gyro[j])
    return q_corr
def angle_estimation_through_madgwick(gyro, acc,delta_t, beta, inertial_quat):
    """
    Estimate the joint rotation angle from IMU data throught Madgwick Filter
    """
    axs_of_rot, delta_theta, q_gyro = angle_axis_quaternion_representation(gyro,delta_t)
    q_corr = tilt_correction_estimation_through_madgwick(q_gyro, acc, delta_t, beta, inertial_quat)
    q_corr_scalar = np.clip(np.array(q_corr)[:, 0], a_min = -1, a_max = 1.0)
    madg_ang = np.rad2deg(2*np.arccos(q_corr_scalar))
    return madg_ang

**Write results to csv files**

In [9]:
def save_in_dest(data, file, dest_key_words, dest_dir, dest_file_name, extension, header):
    for k in range(len(dest_key_words)):
            if file.find(dest_key_words[k]) != -1:
                index = k   
    if file.find('shoulder') != -1 and file.find('flexion') !=-1:
        index = 1
    if file.find('neck') != -1 and file.find('flexion') !=-1 :
        if  file.find('Right_Lateral_flexion') == -1 and file.find('Left_Lateral_flexion') == -1:
            index =4
    path = os.path.join (dest_dir, dest_file_name[index] + extension)
    if not os.path.exists(path):
        with open(path,'w',newline="") as f_in:
            writer = csv.writer(f_in)
            writer.writerow(header)
    with open(path,'a',newline="") as f_in:
        # Write header unchanged
        writer = csv.writer(f_in)
        writer.writerow(data)